<a href="https://colab.research.google.com/github/paulovitor93/medium/blob/main/MEDIUM_Relat%C3%B3rios_Individuais_SCOPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Analysis of SCOPI system reports.
# IFPE - Igarassu
# Paulo Vitor Nascimento de Sousa - Tec. Lab. Man e Sup a Inf. - paulo.sousa1@igarassu.ifpe.edu.br

#######################
###### CHANGELOG ######
#######################
#05/10/2023 --> V_1.0

# Importing the modules
from datetime import datetime
from google.colab import drive
from tabulate import tabulate
import pandas as pd
#import numpy as np
import sys
import os
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Mouting Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Function to sort the reports by number.
def sort_key(file_name):
  return int(file_name.split("_M")[1].split(".xlsx")[0])

# Function to convert columns to date format
def convert_columns_to_date_format(df):
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], infer_datetime_format=True, errors='coerce')

# Organize date with proper format e taking out the 00:00:00
def convert_date_columns_to_format(df, date_format="%d-%m-%Y"):
    for col in date_columns:
        if col in df.columns:
            df[col] = df[col].dt.strftime(date_format)

# Data Columns
date_columns = ['Previsão início', 'Previsão fim', 'Data início', 'Data fim']

#Today's date
today_date = datetime.today().date()

# Unused columns
columns_to_drop = ['Esforço', 'Nível', 'Percentual concluído']

# Choosing Individual or Macroprocess report
user_input = input("I - Individual\nM - Macroprocess\n\nChoose a report type:\n")

# USUÁRIO INDIVIDUAL
# Ensuring case-insensitivity for user input "i" or "I"
if user_input.lower() == "i":

  # Selecting the individual report folder
  path_i = '/content/gdrive/MyDrive/Planejamento_Controle/relatorios/individuais'
  # Getting the list of individual reports in the folder
  reports_i = os.listdir(path_i)
  # Sorting the files in ascending order
  reports_i.sort()

  # Show list of users
  # Adding a blank line for better visualization
  print("")

  # Printing the list of user reports
  # Enumerating the reports in ascending order, starting from 1
  for i, report in enumerate(reports_i, start=1):
    print(f"{i} - {report}")

  # Adding a blank line for better visualization
  print("")
  user_input = input("Select the individual report number: \n")

  # Check if the input is digit
  if user_input.isdigit():
    user_input = int(user_input)

    # Select the number of the report
    if 1 <= user_input <= len(reports_i):
      selected_report = reports_i[user_input -1]

      # Insert the file path into the vaiable
      path = f'{path_i}/{selected_report}'

      # Reading the report
      df = pd.read_excel(path, header=None)

      # Putting the columns name
      df = df.rename(columns=df.iloc[2])

      # Dropping unused columns
      df.drop(columns=columns_to_drop, axis=1, inplace=True)

      # Removing the time and converting into date format
      convert_columns_to_date_format(df)

      # All tasks that are delayed from individual user
      delayed_start = df[(df['Previsão início'].dt.date < today_date) & (df['Data início'].isna())]

      # All tasks that have the end delayed
      delayed_end = df[(df['Previsão início'].dt.date < today_date) & (df['Previsão fim'].dt.date < today_date) & (df['Data início'].notna()) & (df['Data fim'].isna())]

      # All tasks completed
      completed_tasks = df[(~df['Data início'].isna()) & (~df['Data fim'].isna())]

      # All tasks still on-time
      on_time_tasks = df[(df['Previsão início'].dt.date > today_date) & (df['Data início'].isna()) | (df['Previsão início'].dt.date < today_date) & (df['Previsão fim'].dt.date > today_date) & (df['Data início'].notna()) & (df['Data fim'].isna())]

      # All tasks still on-time and started
      on_time_started = on_time_tasks[on_time_tasks['Data início'].notna()]

      # All tasks still on-time but still not started
      on_time_not_started = on_time_tasks[on_time_tasks['Data início'].isna()]

      #All tasks without any date prevision
      no_date = df[(df['Previsão início'].isna()) & (df['Previsão fim'].isna()) & (df['Data fim'].isna()) & (df['Data fim'].isna()) & (df['Responsáveis'].notna()) & (df['Responsáveis'] != "Responsáveis") ]

      # List of all final dataframes do convert the date
      final_df = [delayed_start, delayed_end, completed_tasks, on_time_tasks, on_time_started, on_time_not_started]

      # Converting all data to extract the 00:00:00
      for i in final_df:
        convert_date_columns_to_format(i)

      # Campus
      print(f'\n{df.iloc[1][0].split(">")[3].strip()}\n')

      # Sum of total tasks
      total_tasks = (len(delayed_start) + len(delayed_end) + len(completed_tasks) + len(on_time_tasks) + len(no_date))
      print("Total tasks:", total_tasks)

      # Math to get the percentage of concluded tasks
      t_concluded_tasks = len(completed_tasks)
      concluded_percentage = round(((t_concluded_tasks/total_tasks) * 100), 2)
      print("Total Concluded:", concluded_percentage, "%\n")

      print("Tasks with delayed_start:", len(delayed_start))

      # Table formatting
      table_type = "fancy_grid"
      col_alignment = ("center","left","center","center","center","center", "center")

      # All tasks that are delayed from individual user
      if len(delayed_start) == 0:
        print("There are no tasks with delayed start\n")
      else:
        print(tabulate(delayed_start, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,80]), "\n")

      print("Tasks with delayed end:", len(delayed_end))

      # All tasks that have the end delayed
      if len(delayed_end) == 0:
        print("There are no tasks with delayed end!\n")
      else:
        print(tabulate(delayed_end, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,70]), "\n")

      print("Completed tasks:", len(completed_tasks))

      # All tasks completed
      if len(completed_tasks) == 0:
        print("There are no completed tasks!\n")
      else:
        print(tabulate(completed_tasks, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,60]), "\n")

      # All tasks still good
      print("Tasks on-time:", len(on_time_tasks))
      if len(on_time_tasks) == 0:
        print("There are no tasks on-time!\n")
      elif len(on_time_started) == 0:
        print("Tasks on-time started:", len(on_time_started))
        print("There are no on-time tasks that have already started!")
      else:
        print("On-time started: ", len(on_time_started))
        print(tabulate(on_time_started, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,70]), "\n")

      # All tasks still good but still not started
      if len(on_time_not_started) == 0:
        print("Tasks on-time not started:", len(on_time_not_started))
        print("There are no on-time tasks not started that have not yet started!\n")
      else:
        print("On-time not started: ", len(on_time_not_started))
        print(tabulate(on_time_not_started, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,70]), "\n")

      # All tasks witout date
      print("Tasks without a date: ", len(no_date))
      if len(no_date) == 0:
        print("There are no tasks without a date!")
      else:
        print(tabulate(no_date, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,70]))

    # If there is a wrong report's number
    else:
      print("\nReport Does Not Exist\n")
      sys.exit()
  # If it's not digit
  else:
    print("\nReport Does Not Exist\n")
    sys.exit()

#MACROPROCESSO
elif user_input.lower() == "m":
  path_m = '/content/gdrive/MyDrive/Planejamento_Controle/relatorios/macroprocessos'
  reports_m = os.listdir(path_m)

  # removing the name Responsáveis of the list
  reports_m = [report for report in reports_m if report != '.ipynb_checkpoints']

  # Sorting the list according with the repot's number
  reports_m.sort(key=sort_key)

  # Show list of macroprocesses
  print("")
  for i, report in enumerate(reports_m, start=1):
    print(f"{i} - {report}")

  # Select user by number in the list
  print("")
  user_input = input("Select the macroprocess report number: \n")

  # Checking if a number was inserted correctly
  if user_input.isdigit() and (1 <= int(user_input) <= len(reports_m)):
    user_input = int(user_input)
    selected_report = reports_m[user_input -1]
    path = f'{path_m}/{selected_report}'

    # Reanding the report
    df = pd.read_excel(path, header=None)

    # Putting the columns name
    df = df.rename(columns=df.iloc[2]).drop(df.index[2]).reset_index(drop=True)

    # Dropping unused columns
    df.drop(columns=columns_to_drop, axis=1, inplace=True)

    # Removing the time and converting into date format
    convert_columns_to_date_format(df)

    # List of users in the dataframe
    unique_users = df['Responsáveis'].dropna().unique()

    # Removing the name Responsáveis of the list
    unique_users = [user for user in unique_users if user != 'Responsáveis']

    # Organize the list in alphabetical order
    unique_users = sorted(unique_users)

    # Choosing Individual or Macroprocess report
    user_input = input("\nI - Individual\nC - Complete\n\nSelect the report type for the Macroprocess:\n")

    # USUÁRIO INDIVIDUAL
    if user_input == "I" or user_input == "i":

      # Show list of users
      print("")
      for i, user in enumerate(unique_users, start=1):
        print(f"{i} - {user}")

      # Select user by number in the list
      user_input = input("\nSelecione o usuário desejado:\n")

      # Checking the chosen number and selection the name
      if user_input.isdigit():
        user_input = int(user_input)
        if 1 <= user_input <= len(unique_users):
          selected_user = unique_users[user_input -1]
        else:
          print("\nUsuário Inválido\n")
          sys.exit()
      else:
        print("\nUsuário Inválido\n")
        sys.exit()

      # All tasks that are delayed from user Lincoln
      delayed_start = df[(df['Previsão início'].dt.date < today_date) & (df['Data início'].isna()) & (df['Responsáveis'] == selected_user)]

      # All tasks that have the end delayed
      delayed_end = df[(df['Previsão início'].dt.date < today_date) & (df['Data início'].notna()) & (df['Data fim'].isna()) & (df['Previsão fim'].dt.date < today_date) & (df['Responsáveis'] == selected_user)]
      # All tasks completed
      completed_tasks = df[(df['Data início'].notna()) & (~df['Data fim'].isna()) & (df['Responsáveis'] == selected_user)]

      # All tasks still on-time
      on_time_tasks = df[(df['Previsão início'].dt.date > today_date) & (df['Data início'].isna()) & (df['Responsáveis'] == selected_user) | (df['Previsão início'].dt.date < today_date) & (df['Previsão fim'].dt.date > today_date) & (df['Data início'].notna()) & (df['Data fim'].isna()) & (df['Responsáveis'] == selected_user)]

      # All tasks still on-time and started
      on_time_started = on_time_tasks[on_time_tasks['Data início'].notna()]

      # All tasks still on-time but still not started
      on_time_not_started = on_time_tasks[on_time_tasks['Data início'].isna()]

      # tasks without any dates
      no_date = df[(df['Previsão início'].isna()) & (df['Previsão fim'].isna()) & (df['Data fim'].isna()) & (df['Data fim'].isna()) & (df['Responsáveis'] == selected_user)]

    # MACROPROCESSO
    elif user_input.lower() == "c":

      # All tasks that are delayed from user Lincoln
      delayed_start = df[(df['Previsão início'].dt.date < today_date) & (df['Data início'].isna())]

      # All tasks that have the end delayed
      delayed_end = df[(df['Previsão início'].dt.date < today_date) & (df['Previsão fim'].dt.date < today_date) & (df['Data início'].notna()) & (df['Data fim'].isna())]

      # All tasks completed
      completed_tasks = df[(~df['Data início'].isna()) & (~df['Data fim'].isna())]

      # All tasks still on-time
      on_time_tasks = df[(df['Previsão início'].dt.date > today_date) & (df['Data início'].isna()) | (df['Previsão início'].dt.date < today_date) & (df['Previsão fim'].dt.date > today_date) & (df['Data início'].notna()) & (df['Data fim'].isna())]

      # All tasks still on-time and started
      on_time_started = on_time_tasks[on_time_tasks['Data início'].notna()]

      # All tasks still on-time but still not started
      on_time_not_started = on_time_tasks[on_time_tasks['Data início'].isna()]

      #All tasks without any date prevision
      no_date = df[(df['Previsão início'].isna()) & (df['Previsão fim'].isna()) & (df['Data fim'].isna()) & (df['Data fim'].isna()) & (df['Responsáveis'].notna()) & (df['Responsáveis'] != "Responsáveis") ]

    # if an invalid value is inserted exit and stop application
    else:
        print("\nInvalid Value\n")
        sys.exit()

    # List of all final dataframes do convert the date
    final_df = [delayed_start, delayed_end, completed_tasks, on_time_tasks, on_time_started, on_time_not_started]

    # Converting all data to extract the 00:00:00
    for i in final_df:
      convert_date_columns_to_format(i)

    # Campus
    print("")
    print(df.iloc[1][0].split('>')[3].strip(),"\n")

    # Sum of total tasks
    total_tasks = (len(delayed_start) + len(delayed_end) + len(completed_tasks) + len(on_time_tasks) + len(no_date))
    print("Total tasks:", total_tasks)

    # Math to get the percentage of concluded tasks
    t_completed_tasks = len(completed_tasks)
    percentage_completed = round(((t_completed_tasks/total_tasks) * 100), 2)
    print("Total Completed:", percentage_completed, "%\n")

    print("Tasks with delayed start:", len(delayed_start))

    # Table formatting
    table_type = "fancy_grid"
    col_alignment = ("center","left","center","center","center","center", "center")

    # All tasks that are delayed from user Lincoln
    if len(delayed_start) == 0:
      print("There are no tasks with delayed start !\n")
    else:
      print(tabulate(delayed_start, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,80]), "\n")

    print("Tasks with delayed end:", len(delayed_end))

    # All tasks that have the end delayed
    if len(delayed_end) == 0:
      print("There are no tasks with delayed end!\n")
    else:
      print(tabulate(delayed_end, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,70]), "\n")

    print("Completed_tasks:", len(completed_tasks))
    # All tasks completed
    if len(completed_tasks) == 0:
      print("There are no completed_tasks !\n")
    else:
      print(tabulate(completed_tasks, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,60]), "\n")

    # All tasks still on-time
    print("Tasks on-time: ", len(on_time_tasks), "\n")
    if len(on_time_tasks) == 0:
      print("There are no tasks on-time!\n")
    elif len(on_time_started) == 0:
      print("Tasks on-time started: ", len(on_time_started))
      print("There are no on-time tasks that have already started!\n")
    else:
      print("On-time started: ", len(on_time_started))
      print(tabulate(on_time_started, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,70]), "\n")

    # tasks still on-time but not started
    if len(on_time_not_started) == 0:
      print("On-time tasks not started: ", len(on_time_not_started))
      print("There are no on-time tasks that have not yet been started!\n")
    else:
      print("On-time not started: ", len(on_time_not_started))
      print(tabulate(on_time_not_started, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,70]), "\n")

    # tasks without date
    print("Tasks without date: ", len(no_date))
    if len(no_date) == 0:
      print("There are no tasks without a date!\n")
    else:
      print(tabulate(no_date, headers='keys', tablefmt=table_type, colalign=col_alignment, maxcolwidths=[None,70]), "\n")

  # If the report's number was correctly inserted
  else:
    print("\nNonexistent Report\n")
    sys.exit()
# If the option I - M was correctly inserted
else:
  print("\nInvalid Option\n")
  sys.exit()